In [1]:
import os
import scanpy as sc
import anndata as ad
import glob
# import pyreadr

import h5py

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix


In [2]:
gs1_dir = "data/spotless/gold_standard_1/"

In [3]:
fpaths = sorted(glob.glob(os.path.join(gs1_dir, "*.h5ad")))
fovs = [sc.read_h5ad(name) for name in fpaths]
fovs

[AnnData object with n_obs × n_vars = 9 × 10000
     obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'spot_composition.Endothelial', 'spot_composition.Excitatory.layer.3', 'spot_composition.Excitatory.layer.4', 'spot_composition.Excitatory.layer.II', 'spot_composition.Interneurons', 'spot_composition.Microglia', 'spot_composition.Astrocytes.deep', 'spot_composition.Astrocytes.superficial', 'spot_composition.Interneurons.deep', 'spot_composition.OPC', 'spot_composition.spot_no', 'relative_spot_composition.Endothelial', 'relative_spot_composition.Excitatory.layer.3', 'relative_spot_composition.Excitatory.layer.4', 'relative_spot_composition.Excitatory.layer.II', 'relative_spot_composition.Interneurons', 'relative_spot_composition.Microglia', 'relative_spot_composition.Astrocytes.deep', 'relative_spot_composition.Astrocytes.superficial', 'relative_spot_composition.Interneurons.deep', 'relative_spot_composition.OPC', 'relative_spot_composition.spot_no', 'coordinates.x', 'coordinates.y'
  

In [13]:
obs_cols = sorted(list(set.union(*[set(fov.obs.columns) for fov in fovs])))

In [24]:
for fov in fovs:
    fov.obs = fov.obs.reindex(columns=obs_cols)

1    9871
2    9335
3    6288
4    8474
5    9653
6    6909
7    8725
8    9380
9    4769
Name: nFeature_RNA, dtype: int32

1    7664
2    9791
3    8755
4    7262
5    9143
6    9004
7    8447
8    9831
9    9696
Name: nFeature_RNA, dtype: int32

1    9290
2    9180
3    9732
4    9604
5    9508
6    9530
7    9049
8    9074
9    9290
Name: nFeature_RNA, dtype: int32

1    7264
2    9507
3    8889
4    9307
5    9692
6    9751
7    9784
8    9561
9    9736
Name: nFeature_RNA, dtype: int32

1    6889
2    8716
3    9374
4    9761
5    9140
6    9690
7    9732
8    9451
9    9769
Name: nFeature_RNA, dtype: int32

1    2186
2    9468
3    7680
4    7972
5    8663
6    4466
7    6750
8    8622
9    6558
Name: nFeature_RNA, dtype: int32

1    7715
2    6106
3    8000
4    7199
5    8315
6    3176
7    6374
8    8553
9    6877
Name: nFeature_RNA, dtype: int32

In [28]:
for column in fovs[0].obs.columns:
    display(fovs[0].obs[column])

1     466.019417
2     466.019417
3     466.019417
4    1000.000000
5    1000.000000
6    1000.000000
7    1533.980583
8    1533.980583
9    1533.980583
Name: coordinates.x, dtype: float64

1     466.019417
2    1000.000000
3    1533.980583
4     466.019417
5    1000.000000
6    1533.980583
7     466.019417
8    1000.000000
9    1533.980583
Name: coordinates.y, dtype: float64

1    82549.0
2    47199.0
3    15919.0
4    32359.0
5    61491.0
6    17915.0
7    35974.0
8    48911.0
9     8342.0
Name: nCount_RNA, dtype: float64

1    9871
2    9335
3    6288
4    8474
5    9653
6    6909
7    8725
8    9380
9    4769
Name: nFeature_RNA, dtype: int32

1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: orig.ident, dtype: int32

...

1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: spot_composition.Neural.progenitors, dtype: float64

1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: spot_composition.Neuroblasts, dtype: float64

1    0
2    0
3    0
4    0
5    2
6    0
7    0
8    0
9    0
Name: spot_composition.OPC, dtype: int32

1   NaN
2   NaN
3   NaN
4   NaN
5   NaN
6   NaN
7   NaN
8   NaN
9   NaN
Name: spot_composition.Oligodendrocytes, dtype: float64

1    1.0
2    2.0
3    3.0
4    4.0
5    5.0
6    6.0
7    7.0
8    8.0
9    9.0
Name: spot_composition.spot_no, dtype: float64

In [ ]:

gs1 = ad.concat(fovs, keys=(os.path.basename(f).split(".")[0] for f in fpaths))
gs1

/home/wma/AGrEDA/.venv/lib/python3.8/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_resolver.py:192: DeprecationWarning: Use is_view instead of isview, isview will be removed in the future.
  attr = getattr(var, name)


In [4]:
# adata = sc.read_h5ad("data/pdac/sc_adata/CA001063.h5ad")

In [5]:
# adata

In [6]:
# adata.obs

In [11]:
# result = pyreadr.read_r('data/spotless/gold_standard_1/Eng2019_cortex_svz_fov0.rds')
with h5py.File("data/spotless/gold_standard_1.h5" , "r") as f:
    for name in f:
        gene_names = [str(gene, "utf-8") for gene in f[name]["geneNames"][()]]
        ad.AnnData(csr_matrix(f[name]["counts"][()]), dtype=np.int64)

AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
AnnData object with n_obs × n_vars = 9 × 10000
